In [2]:
import numpy as np
import pandas as pd
import json
import re
import time
import datetime
from urllib.request import urlopen
from urllib.parse import quote   

In [4]:
url = "http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n65000001300000000.html"
response = urlopen(url)
html = response.read().decode('utf-8')
html = html.replace("\t","").replace("\n","").replace("  ","")
html

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta http-equiv="Expires" content="-1"><meta http-equiv="Pragma" content="no-cache"><meta http-equiv="Cache-Control" content="no-cache"><title>107年地方公職人員選舉</title><link href="../css/style.css" rel="stylesheet" type="text/css"><link href="../css/style2.css" rel="stylesheet" type="text/css"><script language="JavaScript" type="text/javascript" src="../js/ftiens4.js"></script><script language="JavaScript" type="text/javascript" src="../js/ua.js"></script><script language="JavaScript" type="text/javascript" src="../js/func.js"></script><script language="JavaScript" type="text/javascript" src="../js/treeC.js"></script><script language="JavaScript" type="text/javascript" src="../js/refresh.js"></script></head><body id="main-type-body" class="main-body"><div id="main-wrap"><div id="main-container"><div id="main-banner">

In [5]:
pat_str1 = '(<tr class="trT"><td>&nbsp;<\/td><td>(\d+)<\/td><td>(.{1,30})<\/td><td>(.{1,2})<\/td><td class="tdAlignRight">(\d+)<\/td><td class="tdAlignRight">([\.\d]+)%<\/td><td>(.{1,30})<\/td><\/tr>)'
pat_re1 = re.compile(pat_str1)

In [6]:
results1 = re.findall(pat_re1, html)
results1

[('<tr class="trT"><td>&nbsp;</td><td>1</td><td>蘇貞昌</td><td>男</td><td class="tdAlignRight">0</td><td class="tdAlignRight">0.00%</td><td>民主進步黨</td></tr>',
  '1',
  '蘇貞昌',
  '男',
  '0',
  '0.00',
  '民主進步黨'),
 ('<tr class="trT"><td>&nbsp;</td><td>2</td><td>侯友宜</td><td>男</td><td class="tdAlignRight">0</td><td class="tdAlignRight">0.00%</td><td>中國國民黨</td></tr>',
  '2',
  '侯友宜',
  '男',
  '0',
  '0.00',
  '中國國民黨')]

In [9]:
pat_str3 = '投開票所數\s+已送/應送:\s+(\d+)\/(\d+)'
pat_re3 = re.compile(pat_str3)

In [10]:
results3 = re.findall(pat_re3, html)
results3

[('0', '142')]

In [75]:
def extract_numbers(url):
    response = urlopen(url)
    html = response.read().decode('utf-8')
    html = html.replace("\t","").replace("\n","").replace("  ","")
    pat_str1 = '(<tr class="trT"><td>&nbsp;<\/td><td>([,\d]+)<\/td><td>(.{1,30})<\/td><td>(.{1,2})<\/td><td class="tdAlignRight">([,\d]+)<\/td><td class="tdAlignRight">([,\.\d]+)%<\/td><td>(.{1,30})<\/td><\/tr>)'
    pat_re1 = re.compile(pat_str1)
    results1 = re.findall(pat_re1, html)
#     print(results1)
    pat_str3 = '投開票所數\s+已送/應送:\s+([,\d]+)\/([,\d]+)'
    pat_re3 = re.compile(pat_str3)
    results3 = re.findall(pat_re3, html)
    results = ([ (int(results1[i][4].replace(",","")), float(results1[i][5].replace(",",""))) for i in range(len(results1))], [int(r) for r in results3[0]])
    return results

In [76]:
results = extract_numbers("http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n65000001100000000.html")
results

([(88, 45.83), (104, 54.17)], [1, 111])

In [25]:
with open("treeFF.js", 'r') as f:
    js_text = f.read()
    pat_str4 = "firAreaName\[\d+\]='(.{1,10})';\s+firAreaID\[(\d+)\]='(\d+)';"
    pat_re4 = re.compile(pat_str4)
    area_codes = re.findall(pat_re4, js_text)
    print(area_codes)
    pat_str5 = "thrAreaName\[(\d+)\]\[(\d+)\]='(.{1,10})';\s+thrAreaID\[\d+\]\[\d+\]=firAreaID\[\d+\]\+'00'\+padding\('(\d+)','(\d+)'\);"
    pat_re5 = re.compile(pat_str5)
    subarea_codes = re.findall(pat_re5, js_text)
    print(subarea_codes)

[('臺北市', '0', '63000'), ('新北市', '1', '65000'), ('桃園市', '2', '68000'), ('臺中市', '3', '66000'), ('臺南市', '4', '67000'), ('高雄市', '5', '64000'), ('新竹縣', '6', '10004'), ('苗栗縣', '7', '10005'), ('彰化縣', '8', '10007'), ('南投縣', '9', '10008'), ('雲林縣', '10', '10009'), ('嘉義縣', '11', '10010'), ('屏東縣', '12', '10013'), ('宜蘭縣', '13', '10002'), ('花蓮縣', '14', '10015'), ('臺東縣', '15', '10014'), ('澎湖縣', '16', '10016'), ('金門縣', '17', '09020'), ('連江縣', '18', '09007'), ('基隆市', '19', '10017'), ('新竹市', '20', '10018'), ('嘉義市', '21', '10020')]
[('0', '0', '松山區', '010', '000'), ('0', '1', '信義區', '020', '000'), ('0', '2', '大安區', '030', '000'), ('0', '3', '中山區', '040', '000'), ('0', '4', '中正區', '050', '000'), ('0', '5', '大同區', '060', '000'), ('0', '6', '萬華區', '070', '000'), ('0', '7', '文山區', '080', '000'), ('0', '8', '南港區', '090', '000'), ('0', '9', '內湖區', '100', '000'), ('0', '10', '士林區', '110', '000'), ('0', '11', '北投區', '120', '000'), ('1', '0', '板橋區', '010', '000'), ('1', '1', '三重區', '020', '000'), ('1', '2', '中和區'

In [26]:
area_code_df = pd.DataFrame(area_codes, columns=["city", "area_index", "area_code"])
area_code_df

,city,area_index,area_code
0,臺北市,0,63000
1,新北市,1,65000
2,桃園市,2,68000
3,臺中市,3,66000
4,臺南市,4,67000
5,高雄市,5,64000
6,新竹縣,6,10004
7,苗栗縣,7,10005
8,彰化縣,8,10007
9,南投縣,9,10008


In [27]:
subarea_code_df = pd.DataFrame(subarea_codes, columns=["area_index", "subarea_index", "district", "subarea_code", "padding"])
subarea_code_df

,area_index,subarea_index,district,subarea_code,padding
0,0,0,松山區,010,000
1,0,1,信義區,020,000
2,0,2,大安區,030,000
3,0,3,中山區,040,000
4,0,4,中正區,050,000
5,0,5,大同區,060,000
6,0,6,萬華區,070,000
7,0,7,文山區,080,000
8,0,8,南港區,090,000
9,0,9,內湖區,100,000


In [28]:
combined_code_df = area_code_df.merge(subarea_code_df, left_on = "area_index", right_on="area_index")
# combined_code_df["padding"] = "00000"
combined_code_df

,city,area_index,area_code,subarea_index,district,subarea_code,padding
0,臺北市,0,63000,0,松山區,010,000
1,臺北市,0,63000,1,信義區,020,000
2,臺北市,0,63000,2,大安區,030,000
3,臺北市,0,63000,3,中山區,040,000
4,臺北市,0,63000,4,中正區,050,000
5,臺北市,0,63000,5,大同區,060,000
6,臺北市,0,63000,6,萬華區,070,000
7,臺北市,0,63000,7,文山區,080,000
8,臺北市,0,63000,8,南港區,090,000
9,臺北市,0,63000,9,內湖區,100,000


In [29]:
combined_code_df["key"] = combined_code_df[["area_code", "subarea_code", "padding"]].apply(lambda x: ''.join(x+"00"), axis=1)
combined_code_df

,city,area_index,area_code,subarea_index,district,subarea_code,padding,key
0,臺北市,0,63000,0,松山區,010,000,63000000100000000
1,臺北市,0,63000,1,信義區,020,000,63000000200000000
2,臺北市,0,63000,2,大安區,030,000,63000000300000000
3,臺北市,0,63000,3,中山區,040,000,63000000400000000
4,臺北市,0,63000,4,中正區,050,000,63000000500000000
5,臺北市,0,63000,5,大同區,060,000,63000000600000000
6,臺北市,0,63000,6,萬華區,070,000,63000000700000000
7,臺北市,0,63000,7,文山區,080,000,63000000800000000
8,臺北市,0,63000,8,南港區,090,000,63000000900000000
9,臺北市,0,63000,9,內湖區,100,000,63000001000000000


In [32]:
all_data = list()
url_root = "http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n"
for line in combined_code_df[["city", "district", "key"]].values:
    all_data.append( (line[0], line[1], url_root+line[2]+".html") )
all_url_df = pd.DataFrame(all_data, columns=["city", "district", "url"])

In [33]:
all_url_df

,city,district,url
0,臺北市,松山區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
1,臺北市,信義區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
2,臺北市,大安區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
3,臺北市,中山區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
4,臺北市,中正區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
5,臺北市,大同區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
6,臺北市,萬華區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
7,臺北市,文山區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
8,臺北市,南港區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...
9,臺北市,內湖區,http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n63000...


In [34]:
output_filename = "data/mayoral_results_url.csv"
all_url_df.to_csv(output_filename)

In [79]:
def crawler():
    
    all_data = list()
    url_root = "http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n"

    count = 1
    print("processing loop {}".format(count))
    t1 = time.time()
    for line in combined_code_df[["city", "district", "key"]].values:
        url = url_root+line[2]+".html"
        try:
            results = extract_numbers(url)
    #         print(line)
    #         print(results)
            for no, sub_result in enumerate(results[0]):
                t = datetime.datetime.now()
                t = t + datetime.timedelta(hours=16, minutes=0)
                all_data.append( (  line[0], line[1], no+1,
                                    sub_result[0], sub_result[1],
                                    results[1][0], results[1][1], t.strftime("%Y%m%d-%H%M%S") ) )
        except Exception as ex:
            print(url)
            print(ex)
    t2 = time.time()
    print("done mayoral in {} s".format(t2-t1))
    referendum_df = pd.DataFrame(all_data, columns=["縣市", "行政區", "候選人", "得票數", "得票率",
                                                 "已送投開票所數", "應送投開票所數", "時間"])
    output_filename = "data/mayoral_results_latest.csv"
    referendum_df.to_csv(output_filename)
    t = datetime.datetime.now()
    t = t + datetime.timedelta(hours=16, minutes=0)
    output_filename = "data/mayoral_results_{}.csv".format(t.strftime("%Y%m%d-%H%M%S"))
    referendum_df.to_csv(output_filename)

    count = count+1

In [ ]:
while True:
    crawler()

processing loop 1
done mayoral in 66.22770881652832 s
processing loop 1


time.struct_time(tm_year=2018, tm_mon=11, tm_mday=24, tm_hour=8, tm_min=44, tm_sec=32, tm_wday=5, tm_yday=328, tm_isdst=0)

('0', '0', '0', '0')